# Generating ML XS, ACE Files, Benchmark Inputs and Serpent Scripts

In [1]:
import pandas as pd
import os
import logging

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.mode.chained_assignment = None  # default='warn'

import sys
# This allows us to import the nucml utilities
sys.path.append("..")


import nucml.datasets as nuc_data
import nucml.ace.data_utilities as ace_utils
import nucml.model.utilities as model_utils

In [2]:
figure_dir = "Figures/"

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
import importlib
importlib.reload(nuc_data)
importlib.reload(ace_utils)
importlib.reload(model_utils)
print("Finish re-loading scripts.")

Finish re-loading scripts.


In [5]:
import nucml.exfor.data_utilities as exfor_utils
import nucml.general_utilities as gen_utils
import nucml.config as config
importlib.reload(config)
importlib.reload(gen_utils)
importlib.reload(exfor_utils)

<module 'nucml.exfor.data_utilities' from '..\\nucml\\exfor\\data_utilities.py'>

## Loading Datasets Dataset

In [6]:
# LOADING DATASET
df_b0, _, _, _, _, to_scale_b0, _ = nuc_data.load_exfor(pedro=True, basic=0, normalize=False)
df_b1, _, _, _, _, to_scale_b1, _ = nuc_data.load_exfor(pedro=True, basic=1, normalize=False)
df_b2, _, _, _, _, to_scale_b2, _ = nuc_data.load_exfor(pedro=True, basic=2, normalize=False)
df_b3, _, _, _, _, to_scale_b3, _ = nuc_data.load_exfor(pedro=True, basic=3, normalize=False)
df_b4, _, _, _, _, to_scale_b4, _ = nuc_data.load_exfor(pedro=True, basic=4, normalize=False)

## Decision Tree

In [9]:
dt_ml_ace_dir_b0 = "ml/DT_B0/"
dt_ml_ace_dir_b1 = "ml/DT_B1/"
dt_ml_ace_dir_b2 = "ml/DT_B2/"
dt_ml_ace_dir_b3 = "ml/DT_B3/"
dt_ml_ace_dir_b4 = "ml/DT_B4/"

In [11]:
results_b0 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB0.csv").sort_values(by="max_depth")
results_b1 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB1.csv").sort_values(by="max_depth")
results_b2 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB2.csv").sort_values(by="max_depth")
results_b3 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB3.csv").sort_values(by="max_depth")
results_b4 = pd.read_csv("../ML_EXFOR_neutrons/2_DT/dt_resultsB4.csv").sort_values(by="max_depth")

results_b0 = results_b0[results_b0.normalizer == "none"]

In [21]:
# filter down to just three for testing purposes:
dt_results = model_utils.get_best_models_df(results_b0).iloc[[0]]

In [22]:
BENCHMARK_NAME = "U233_MET_FAST_001"

In [ ]:
BENCHMARK_NAME_2 = "U233_MET_FAST_002"

In [17]:
ace_utils.generate_bench_ml_xs(df_b0, dt_results, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir_b0, reset=True)

In [16]:
BENCHMARK_NAME = "U233_MET_FAST_001"

ace_utils.generate_bench_ml_xs(df_b0, dt_results_b0, BENCHMARK_NAME, to_scale_b0, dt_ml_ace_dir_b0, reset=True)
# ace_utils.generate_bench_ml_xs(df_b1, dt_results_b1, BENCHMARK_NAME, to_scale_b1, dt_ml_ace_dir_b1, reset=True)
# ace_utils.generate_bench_ml_xs(df_b2, dt_results_b2, BENCHMARK_NAME, to_scale_b2, dt_ml_ace_dir_b2, reset=True)
# ace_utils.generate_bench_ml_xs(df_b3, dt_results_b3, BENCHMARK_NAME, to_scale_b3, dt_ml_ace_dir_b3, reset=True)
# ace_utils.generate_bench_ml_xs(df_b4, dt_results_b4, BENCHMARK_NAME, to_scale_b4, dt_ml_ace_dir_b4, reset=True)

NameError: name 'dt_results_b0' is not defined

In [18]:
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b0, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b1, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b2, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b3, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)
ace_utils.generate_serpent_bash(dt_ml_ace_dir_b4, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

### K-Nearest-Neighbors

In [9]:
knn_ml_ace_dir = "ml/KNN_B0/"

In [10]:
knn_results_filepath = "../ML_EXFOR_neutrons/1_KNN/knn_results_B0.csv"
knn_results = pd.read_csv(knn_results_filepath)
knn_results["scale_energy"] = knn_results.run_name.apply(lambda x: True if "v2" in x else False)
# knn_results = model_utils.get_best_models_df(knn_results)
knn_results

,id,distance_metric,mt_strategy,normalizer,train_mae,train_mse,train_evs,train_mae_m,train_r2,val_mae,val_mse,val_evs,val_mae_m,val_r2,test_mae,test_mse,test_evs,test_mae_m,test_r2,model_path,training_time,scaler_path,run_name,scale_energy
0,13,euclidean,one_hot,standard,0.025880,0.010936,0.985814,0.0,0.985814,0.118292,0.062828,0.915687,0.042361,0.915686,0.118816,0.063698,0.914577,0.042778,0.914573,E:\ML_Models_EXFOR\KNN_B0\k13_distance_euclide...,3671.971000,E:\ML_Models_EXFOR\KNN_B0\k13_distance_euclide...,k13_distance_euclidean_standard_one_hot_B0_v1,False
1,9,euclidean,one_hot,standard,0.025931,0.010999,0.985734,0.0,0.985734,0.118136,0.062010,0.917756,0.041772,0.917756,0.118899,0.064408,0.913698,0.041733,0.913698,E:\ML_Models_EXFOR\KNN_B0\k9_distance_euclidea...,4992.854609,E:\ML_Models_EXFOR\KNN_B0\k9_distance_euclidea...,k9_distance_euclidean_standard_one_hot_B0_v1,False
2,1,euclidean,one_hot,standard,0.029060,0.021444,0.972555,0.0,0.972555,0.140604,0.090440,0.885060,0.047921,0.885059,0.140549,0.091576,0.882881,0.047418,0.882880,E:\ML_Models_EXFOR\KNN_B0\k1_distance_euclidea...,6258.326797,E:\ML_Models_EXFOR\KNN_B0\k1_distance_euclidea...,k1_distance_euclidean_standard_one_hot_B0_v1,False
3,5,euclidean,one_hot,standard,0.026062,0.011286,0.985346,0.0,0.985346,0.119661,0.066503,0.911902,0.041176,0.911900,0.119958,0.067081,0.911042,0.041299,0.911040,E:\ML_Models_EXFOR\KNN_B0\k5_distance_euclidea...,6381.527638,E:\ML_Models_EXFOR\KNN_B0\k5_distance_euclidea...,k5_distance_euclidean_standard_one_hot_B0_v1,False
4,17,euclidean,one_hot,standard,0.025844,0.010874,0.985899,0.0,0.985899,0.119776,0.064781,0.912524,0.043461,0.912522,0.119740,0.064122,0.913774,0.043261,0.913773,E:\ML_Models_EXFOR\KNN_B0\k17_distance_euclide...,6478.077649,E:\ML_Models_EXFOR\KNN_B0\k17_distance_euclide...,k17_distance_euclidean_standard_one_hot_B0_v1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,4,euclidean,one_hot,minmax,0.026206,0.011535,0.985050,0.0,0.985050,0.119090,0.063827,0.916232,0.040870,0.916232,0.118699,0.064212,0.914864,0.040746,0.914864,E:\ML_Models_EXFOR\KNN_B0\k4_distance_euclidea...,52908.405799,E:\ML_Models_EXFOR\KNN_B0\k4_distance_euclidea...,k4_distance_euclidean_minmax_one_hot_B0_v2,True
276,8,manhattan,one_hot,minmax,0.025926,0.011033,0.985687,0.0,0.985687,0.116317,0.060090,0.920073,0.041105,0.920073,0.115661,0.058934,0.921851,0.040820,0.921849,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,45342.842937,E:\ML_Models_EXFOR\KNN_B0\k8_distance_manhatta...,k8_distance_manhattan_minmax_one_hot_B0_v2,True
277,12,manhattan,one_hot,minmax,0.025839,0.010890,0.985876,0.0,0.985876,0.115830,0.059749,0.920521,0.041403,0.920521,0.115695,0.059854,0.920381,0.041307,0.920381,E:\ML_Models_EXFOR\KNN_B0\k12_distance_manhatt...,48068.513939,E:\ML_Models_EXFOR\KNN_B0\k12_distance_manhatt...,k12_distance_manhattan_minmax_one_hot_B0_v2,True
278,16,manhattan,one_hot,minmax,0.025825,0.010862,0.985911,0.0,0.985911,0.115843,0.059055,0.921012,0.041857,0.921012,0.116128,0.059337,0.921421,0.042361,0.921421,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,46104.734946,E:\ML_Models_EXFOR\KNN_B0\k16_distance_manhatt...,k16_distance_manhattan_minmax_one_hot_B0_v2,True


In [11]:
BENCHMARK_NAME = "U233-MET-FAST-001"
ace_utils.generate_bench_ml_xs(df_b0, knn_results, BENCHMARK_NAME, to_scale_b0, knn_ml_ace_dir, reset=True)

In [13]:
ace_utils.generate_serpent_bash(knn_ml_ace_dir, BENCHMARK_NAME, benchmark=BENCHMARK_NAME)

# XGBoost

In [11]:
# all_serpent_files = []

# for root, _, files in os.walk("ml/DT_B0"):
#     for file in files:
#         if "U233_MET_FAST_001_001" in root:
#             all_serpent_files.append(os.path.abspath(os.path.join(root, file)))